In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import openpyxl
from string import Template
import pandas as pd

In [2]:
def get_cik(tick):
    elink = Template(
        'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=$ticker'
        '&type=&dateb=&owner=exclude&count=100&output=xml')
    filled_elink = elink.substitute(ticker=tick)
    res = requests.get(filled_elink)
    res.raise_for_status()
    elinksoup = BeautifulSoup(res.text, 'lxml')
    cik_number = elinksoup.cik.string
    return cik_number

In [3]:
def get_filing_dates(cik):
    filing_dates = []
    hist_search = Template(
        'https://www.sec.gov/cgi-bin/srch-edgar'
        '?text=cik%3D%28$ciknum%29+form-type%3D%2810-q*+OR+10-k*%29&first=2010&last=2018')
    filled_search = hist_search.substitute(ciknum=cik)
    res = requests.get(filled_search)
    res.raise_for_status()
    hist_soup = BeautifulSoup(res.text, 'lxml')
    hist_table = hist_soup.find_all('table')[4]
    page_links = hist_table.select('tr > td:nth-of-type(3) > a[href*="index"]')
    for el in page_links:
        filing_dates.append(scrap_dates(el))
    return filing_dates

In [4]:
def scrap_dates(element):
        plink_filled = 'https://www.sec.gov' + element.get('href')
        index_page = requests.get(plink_filled)
        index_page_bs = BeautifulSoup(index_page.text, 'lxml')
        form_name = index_page_bs.select('div > strong')[0].text
        filing_date = index_page_bs.select('[class~="info"]')[0].text
        end_period = index_page_bs.select('[class~="info"]')[3].text
        ele_dates = {
            "form":form_name,
            "filed":filing_date,
            "period":end_period
        }
        return ele_dates

In [3]:
# load the worksheet and select the sheet with the tickers
worksheet = openpyxl.load_workbook("worksheet_macro.xlsm")
worksheet.sheetnames
wstop = worksheet['Top50']

In [4]:
# iterate through the rows to get the tickers of all the companies
ticker = []
for row in range(0, 50):
    ticker.append(wstop['C' + (str(row+2))].value.strip())

In [82]:
ticker[4]

'NFLX'

In [5]:
get_cik('NFLX')

'0001065280'

In [11]:
n_dates = get_filing_dates(get_cik('AMZN'))

In [17]:
list(filter(lambda filing: filing['period'] == '2013-03-31', n_dates))[0]['filed']

'2013-04-26'